In [1]:
import torch
import torchvision
import numpy as np

In [2]:
class Collector:
    def __init__(self):
        self.ret = []
    
    def filter_(self, m):
        return isinstance(m, torch.nn.Conv2d) and \
                m.stride == (1, 1) and \
                m.kernel_size == (3, 3)

    def hook(self, m, in_, out_):
        if self.filter_(m):
            tmp = (m, in_[0].shape[2:], out_.shape[2:])
            self.ret.append(tmp)

    def register(self, m):
        if not m._modules:
            m.register_forward_hook(self.hook)

In [3]:
m = torchvision.models.resnet.resnet50(True)
inp = torch.rand(1, 3, 512, 512)
c = Collector()
m.apply(c.register)
m(inp); None

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/ubuntu/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [25]:
tmp = []

with open('resnet50_conv3x3_fmt.txt', 'w') as f:
    print(len(c.ret), file=f)
    for i, (m, *_) in enumerate(c.ret, 1):
        w = m.weight.detach().numpy()
        tmp.append(w.reshape(-1))
        print(*w.shape, file=f)

np.concatenate(tmp).tofile('resnet50_conv3x3_dat.bin')

In [37]:
import struct
with open('resnet50_conv3x3_dat.bin', 'rb') as f:
    dat = f.read(64*64*3*3*4)
    print(struct.unpack_from('@fffffffff', dat, 0))
    print(c.ret[0][0].weight[0,0,0,:])

(2.0918651255641407e-09, -1.4151557703456774e-09, 6.584983669455369e-09, 4.792239494122441e-09, 3.2818754380770088e-09, 9.428415825851744e-10, 6.476260860921457e-09, -3.0155689056954316e-09, -2.3958237616028555e-09)
tensor([ 2.0919e-09, -1.4152e-09,  6.5850e-09], grad_fn=<SliceBackward>)


In [30]:
struct?